# Example usage

To use `cat2cat` in a project:

Unifying an inconsistent coded categorical variable in a panel/longtitudal dataset.
There is offered the cat2cat procedure to map a categorical variable according to a mapping (transition) table between two different time points. The mapping (transition) table should to have a candidate for each category from the targeted for an update period. The main rule is to replicate the observation if it could be assigned to a few categories, then using simple frequencies or statistical methods to approximate probabilities of being assigned to each of them.

This algorithm was invented and implemented in the paper by (Nasinski, Majchrowska and Broniatowska (2020) doi:10.24425/cejeme.2020.134747).

## Graph - cat2cat procedure

The graphs present how the `cat2cat` function (and the underlying procedure) works, in this case under a panel dataset without the unique identifiers and only two periods.

![Backward Mapping](https://raw.githubusercontent.com/Polkas/cat2cat/master/man/figures/back_nom.png)

![Forward Mapping](https://raw.githubusercontent.com/Polkas/cat2cat/master/man/figures/for_nom.png)

### Load example data

In [1]:
# cat2cat datasets
from cat2cat.datasets import load_trans, load_occup, load_verticals
trans = load_trans()
occup = load_occup()
verticals = load_verticals()

### Low-level functions

In [2]:

# Low-level functions
from cat2cat.mappings import get_mappings, get_freqs, cat_apply_freq

mappings = get_mappings(trans)
codes_new = occup.code[occup.year == 2010].map(str).values
freqs = get_freqs(codes_new)
mapp_new_p = cat_apply_freq(mappings["to_new"], freqs)
# base period mappings and their frequencies probabilities
(mappings["to_new"]['3481'], mapp_new_p['3481'])

(['441401', '441402', '441403', '441490'], [0.0, 0.6, 0.0, 0.4])

### cat2cat procedure - one iteration

In [3]:
from cat2cat import cat2cat
from cat2cat.dataclass import cat2cat_data, cat2cat_mappings, cat2cat_ml

from pandas import DataFrame, concat

In [4]:
o_2006 = occup.loc[occup.year == 2006, :].copy()
o_2008 = o_old = occup.loc[occup.year == 2008, :].copy()
o_2010 = o_new = occup.loc[occup.year == 2010, :].copy()
o_2012 = occup.loc[occup.year == 2012, :].copy()

data = cat2cat_data(o_old, o_new, "code", "code", "year")
mappings = cat2cat_mappings(trans, "backward")

c2c = cat2cat(data, mappings)
data_final = concat([c2c["old"], c2c["new"]])

sub_cols = ["id", "edu", "code", "year", "index_c2c", "g_new_c2c", "rep_c2c", "wei_naive_c2c", "wei_freq_c2c"]
data_final.groupby(["year"]).sample(5).loc[:, sub_cols]

,id,edu,code,year,index_c2c,g_new_c2c,rep_c2c,wei_naive_c2c,wei_freq_c2c
222409,33326,6,8162,2008,16785,818202,7,0.142857,0.076923
187289,30616,5,5159,2008,14075,541904,23,0.043478,0.000000
138020,26856,2,2131,2008,10315,252101,19,0.052632,0.083333
57209,20792,1,2231,2008,4251,221233,70,0.014286,0.000000
162525,28767,1,2321,2008,12226,235601,34,0.029412,0.000000
34898,34899,2,121101,2010,1135,121101,1,1.000000,1.000000
45008,45009,1,241305,2010,11245,241305,1,1.000000,1.000000
47512,47513,1,251904,2010,13749,251904,1,1.000000,1.000000
42733,42734,1,213203,2010,8970,213203,1,1.000000,1.000000
49903,49904,3,522301,2010,16140,522301,1,1.000000,1.000000


### With ML

In [5]:
from sklearn.neighbors import KNeighborsClassifier

ml = cat2cat_ml(
    o_new, 
    "code", 
    ["salary", "age", "edu"], 
    [KNeighborsClassifier()]
)

c2c = cat2cat(data, mappings, ml)
data_final = concat([c2c["old"], c2c["new"]])

sub_cols = ["id", "year", "wei_naive_c2c", "wei_freq_c2c", "wei_KNeighborsClassifier_c2c"]
data_final.groupby(["year"]).sample(5).loc[:, sub_cols]

,id,year,wei_naive_c2c,wei_freq_c2c,wei_KNeighborsClassifier_c2c
143076,27292,2008,0.076923,0.000000,0.0
95323,23481,2008,0.029412,0.025974,0.0
43382,19734,2008,0.125000,0.077236,0.0
10629,17237,2008,0.111111,0.000000,0.0
16473,17645,2008,0.041667,0.002404,0.0
51007,51008,2010,1.000000,1.000000,1.0
47406,47407,2010,1.000000,1.000000,1.0
49151,49152,2010,1.000000,1.000000,1.0
47979,47980,2010,1.000000,1.000000,1.0
39184,39185,2010,1.000000,1.000000,1.0


With 4 periods, one mapping table and backward direction:

In [6]:
from cat2cat.cat2cat_utils import dummy_c2c

o_2006 = occup.loc[occup.year == 2006, :].copy()
o_2008 = occup.loc[occup.year == 2008, :].copy()
o_2010 = occup.loc[occup.year == 2010, :].copy()
o_2012 = occup.loc[occup.year == 2012, :].copy()


data = cat2cat_data(o_2008, o_2010, "code", "code", "year")
mappings = cat2cat_mappings(trans, "backward")

occup_back_2008_2010 = cat2cat(data, mappings)
data = cat2cat_data(
    o_2006, occup_back_2008_2010["old"], 
    "code", "g_new_c2c", "year"
)
occup_back_2006_2008 = cat2cat(data, mappings)

o_2006_n = occup_back_2006_2008["old"]
o_2008_n = occup_back_2006_2008["new"] # or occup_back_2008_2010["old"]
o_2010_n = occup_back_2008_2010["new"]
o_2012_n = dummy_c2c(o_2012, "code")

data_final = concat([o_2006_n, o_2008_n, o_2010_n, o_2012_n])

sub_cols = ["id", "edu", "code", "year", "index_c2c",
 "g_new_c2c", "rep_c2c", "wei_naive_c2c", "wei_freq_c2c"]
data_final.groupby(["year"]).sample(5).loc[:, sub_cols]

,id,edu,code,year,index_c2c,g_new_c2c,rep_c2c,wei_naive_c2c,wei_freq_c2c
36823,3498,4,3112,2006,3497,311206,18,0.055556,0.093750
160915,12525,2,1212,2006,12524,112009,16,0.062500,0.027027
186092,14506,6,7321,2006,14505,731408,10,0.100000,0.133333
28039,2682,4,4211,2006,2681,523002,4,0.250000,0.510638
2826,277,3,2419,2006,276,242204,46,0.021739,0.011574
216193,32834,6,9132,2008,16293,941202,18,0.055556,0.001294
158569,28431,6,4141,2008,11890,441201,5,0.200000,0.222222
9201,17110,4,2241,2008,569,222207,24,0.041667,0.000000
1558,16647,2,2419,2008,106,243104,46,0.021739,0.004630
93389,23336,6,8312,2008,6795,831201,12,0.083333,0.106796


#### Prune - prune_c2c

Pruning which could be useful after the mapping process, the custom prune_fun is provided by the end user.
The prune_fun is a function to process a 1D-array of weights (float) and return a 1D-array of boolean of the same length. The weighs will be reweighted automatically to still to sum to one per each original observation.

- non-zero - lambda x: x > 0
- highest1 - lambda x: arange(len(x)) == argmax(x)
- highest - lambda x: x == max(x)

In [7]:
from cat2cat.cat2cat_utils import prune_c2c
from numpy import arange, argmax

# prune_c2c
# highest1 leave only one observation with the highest probability for each orginal one
(o_2006_n.shape[0], 
 prune_c2c(o_2006_n, lambda x: arange(len(x)) == argmax(x)).shape[0])

(212518, 16540)

### Direct match

It is important to set the `id_var` argument as then we merging categories 1 to 1
for this identifier which exists in both periods.

In [8]:
vert_old = verticals.loc[verticals["v_date"] == "2020-04-01", :]
vert_new = verticals.loc[verticals["v_date"] == "2020-05-01", :]

## extract mapping (transition) table from data using identifier
trans_v = vert_old.merge(vert_new, on = "ean", how = "inner")\
.loc[:, ["vertical_x", "vertical_y"]]\
.drop_duplicates()

In [9]:
data = cat2cat_data(
  old = vert_old, new = vert_new, id_var = "ean", 
  cat_var_old = "vertical", cat_var_new = "vertical", time_var = "v_date"
)
mappings = cat2cat_mappings(trans_v, "backward")

verts = cat2cat(
  data = data,
  mappings = mappings
)

data_final = concat([verts["old"], verts["new"]])
data_final.groupby(["v_date"]).sample(5)

,ean,vertical,sales,v_date,index_c2c,g_new_c2c,rep_c2c,wei_naive_c2c,wei_freq_c2c
47,90000048,Clothes,97.907028,2020-04-01,27,Clothes,1,1.0,1.0
20,90000017,Sport,91.490275,2020-04-01,16,Sport,1,1.0,1.0
20,90000021,Home,102.966520,2020-04-01,0,Supermarket,1,1.0,1.0
40,90000041,Kids1,123.018540,2020-04-01,20,Kids,1,1.0,1.0
19,90000016,Fashion,91.422618,2020-04-01,15,Fashion,1,1.0,1.0
158,90000079,Home,79.879837,2020-05-01,58,Home,1,1.0,1.0
154,90000075,Automotive1,73.972392,2020-05-01,54,Automotive1,1,1.0,1.0
112,90000033,Books,81.925987,2020-05-01,12,Books,1,1.0,1.0
100,90000021,Supermarket,86.656355,2020-05-01,0,Supermarket,1,1.0,1.0
148,90000069,Fashion,75.732352,2020-05-01,48,Fashion,1,1.0,1.0


### Direct match with ML

In [10]:
ml = cat2cat_ml(
    vert_old, 
    "vertical", 
    ["sales"], 
    [KNeighborsClassifier()]
)

verts_ml = cat2cat(
  data = data,
  mappings = mappings,
  ml = ml
)

data_final = concat([verts_ml["old"], verts_ml["new"]])
data_final.groupby(["v_date"]).sample(5)

,ean,vertical,sales,v_date,index_c2c,g_new_c2c,rep_c2c,wei_naive_c2c,wei_freq_c2c,wei_KNeighborsClassifier_c2c
97,90000098,Kids1,94.204355,2020-04-01,77,Kids,1,1.0,1.0,1.0
65,90000066,Fashion,101.943413,2020-04-01,45,Fashion,1,1.0,1.0,1.0
85,90000086,Clothes,83.298969,2020-04-01,65,Clothes,1,1.0,1.0,1.0
87,90000088,Kids1,83.214062,2020-04-01,67,Kids,1,1.0,1.0,1.0
38,90000039,Health,99.849505,2020-04-01,18,Health,1,1.0,1.0,1.0
141,90000062,Sport,102.571438,2020-05-01,41,Sport,1,1.0,1.0,1.0
195,90000116,Books,94.701973,2020-05-01,95,Books,1,1.0,1.0,1.0
112,90000033,Books,81.925987,2020-05-01,12,Books,1,1.0,1.0,1.0
167,90000088,Kids,67.695806,2020-05-01,67,Kids,1,1.0,1.0,1.0
135,90000056,Electronics,75.327059,2020-05-01,35,Electronics,1,1.0,1.0,1.0
